In [9]:
from bs4 import BeautifulSoup
from requests import get
import pandas as pd

In [10]:
headers = ({'User-Agent':
            'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit\
            /537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36'})
data_list = []
for page in range(1,21):
    base_url = ["https://www.autoscout24.de/lst/?sort=standard&desc=0&offer=N%2CU&fuel=B%2CD&ustate=N%2CU&size=20&page=" +str(page) + "&cy=D&kmto=100000&fregfrom=2010&body=1&atype=C&fc=24&qry=&",
                "https://www.autoscout24.de/lst/?sort=standard&desc=0&offer=N%2CU&fuel=B%2CD&ustate=N%2CU&size=20&page=" +str(page) + "&cy=D&kmto=100000&fregfrom=2010&body=&atype=C&fc=24&qry=&",
                "https://www.autoscout24.de/lst/?sort=standard&desc=0&offer=N%2CU&fuel=B%2CD&ustate=N%2CU&size=20&page=" +str(page) + "&cy=D&kmto=100000&fregfrom=2010&body=3&atype=C&fc=24&qry=&",
                "https://www.autoscout24.de/lst/?sort=standard&desc=0&offer=N%2CU&fuel=B%2CD&ustate=N%2CU&size=20&page=" +str(page) + "&cy=D&kmto=100000&fregfrom=2010&body=4&atype=C&fc=24&qry=&",
                "https://www.autoscout24.de/lst/?sort=standard&desc=0&offer=N%2CU&fuel=B%2CD&ustate=N%2CU&size=20&page=" +str(page) + "&cy=D&kmto=100000&fregfrom=2010&body=5&atype=C&fc=24&qry=&",
                "https://www.autoscout24.de/lst/?sort=standard&desc=0&offer=N%2CU&fuel=B%2CD&ustate=N%2CU&size=20&page=" +str(page) + "&cy=D&kmto=100000&fregfrom=2010&body=6&atype=C&fc=24&qry=&",
                "https://www.autoscout24.de/lst/?sort=standard&desc=0&offer=N%2CU&fuel=B%2CD&ustate=N%2CU&size=20&page=" +str(page) + "&cy=D&kmto=100000&fregfrom=2010&body=12&atype=C&fc=24&qry=&"
               ]
    for idx in range(len(base_url)):
        response = get(base_url[idx],headers=headers)
        #print(response)
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')
        data_list.append(soup.find_all('div', attrs={'class': 'cldt-summary-full-item-main'}))
#print(data_list)

In [59]:
model_info = []
price_info = []
general_info = []
for item in data_list:
    for auto in item:
        model_info.append(auto.find_all('div', attrs={'class': 'cldt-summary-title'}))
        price_info.append(auto.find_all('div', attrs={'class': 'cldt-summary-payment'}))
        general_info.append(auto.find_all('div', attrs={'class': 'cldt-summary-vehicle-data'}))

#print(len(model_info))
#print(len(price_info))
#print(len(general_info))
#print(model_info)
#print(price_info)
#print(general_info)

In [67]:
def convert_list_to_string(org_list, seperator=' '):
    """ Convert list to string, by joining all item in list with given separator.
        Returns the concatenated string """
    return seperator.join(org_list)

In [68]:
brands = []
models = []
for model in model_info:
    for m in model:
        md = m.find_all("h2", attrs={'class': "cldt-summary-makemodel sc-font-bold sc-ellipsis"})[0].text.strip()
        if len(md.split()) >= 3:
            if 'Land Rover' in md:
                brands.append("Land Rover")
                temp = md.split()[1:]
                mdl = convert_list_to_string(temp)
                models.append(mdl)
            else:        
                brands.append(md.split()[0])
                temp = md.split()[1:]
                mdl = convert_list_to_string(temp)
                models.append(mdl)
            
                
        
        else:
            brands.append(md.split()[0])
            temp = md.split()[1:]
            mdl = convert_list_to_string(temp)
            models.append(mdl)
#print(brands)
#print(models)

In [69]:
prices = []
for price in price_info:
    for p in price:
        pr = p.find_all("span", attrs={"class": "cldt-price sc-font-xl sc-font-bold"})[0].text.strip()
        prc = pr.split()[1].replace("-","").replace(",", "").replace(".","")
        prices.append(float(prc))
#print(prices)

In [70]:
first_reg_year = []
milleage = []
transmission = []
fuel_type = []
engine_power = []
condition = []
        
for auto in general_info:
    for spec in auto:
        prod = spec.find_all('li')[1].text.strip()
        first_reg_year.append(prod)
                         
        ml = spec.find_all('li')[0].text.strip().replace(".", "").split()[0]
        if ml == "-":
            ml = '100'
            milleage.append(ml)
            
        else:
            milleage.append(ml)
        
        fuel_type.append(spec.find_all('li', attrs={'class': "summary_item_no_bottom_line"})[0].text.strip())
        transmission.append(spec.find_all('li')[5].text.strip())
        condition.append(spec.find_all('li')[3].text.strip())
        
        enp = spec.find_all('li')[2].text.strip().replace("(","").replace(")","").split()[2]
        if enp == "-":
            enp = '100'
            engine_power.append(float(enp))
            
        else:
            engine_power.append(float(enp))
      
#print(fuel_type)
#print(prod_year)
#print(milleage)
#print(engine_power)
#print(transmission)
#print(condition)


In [71]:
data = pd.DataFrame({'Brand': brands, 'Model': models, "Fuel type": fuel_type, "Transmission": transmission,'Conditon': condition , "First Registration Year": first_reg_year, "Milleage (kms)":milleage, "Engine Power (PS)": engine_power, "Price (EUR)":prices})

In [72]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2800 entries, 0 to 2799
Data columns (total 9 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Brand                    2800 non-null   object 
 1   Model                    2800 non-null   object 
 2   Fuel type                2800 non-null   object 
 3   Transmission             2800 non-null   object 
 4   Conditon                 2800 non-null   object 
 5   First Registration Year  2800 non-null   object 
 6   Milleage (kms)           2800 non-null   object 
 7   Engine Power (PS)        2800 non-null   float64
 8   Price (EUR)              2800 non-null   float64
dtypes: float64(2), object(7)
memory usage: 197.0+ KB


In [73]:
for el1 in data['Fuel type']:
    if "1 kWh/100 km (komb)" in el1:
        indeces = data.index[data['Fuel type'] == el1].tolist()
        
data.drop(indeces, inplace=True)

In [74]:
for el3 in data['Transmission']:
    if el3 == "-/- (Getriebeart)":
        ind = data.index[data['Transmission'] == el3].tolist()
        
data.drop(ind, inplace=True)

In [75]:
lst = data.index[data['First Registration Year'] == '-/- (Erstzulassung)'].tolist()
for el2 in lst:
    data.at[el2, 'First Registration Year'] = '09/2020'

In [76]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2780 entries, 0 to 2799
Data columns (total 9 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Brand                    2780 non-null   object 
 1   Model                    2780 non-null   object 
 2   Fuel type                2780 non-null   object 
 3   Transmission             2780 non-null   object 
 4   Conditon                 2780 non-null   object 
 5   First Registration Year  2780 non-null   object 
 6   Milleage (kms)           2780 non-null   object 
 7   Engine Power (PS)        2780 non-null   float64
 8   Price (EUR)              2780 non-null   float64
dtypes: float64(2), object(7)
memory usage: 297.2+ KB


In [77]:
data.describe()

,Engine Power (PS),Price (EUR)
count,2780.000000,2780.000000
mean,127.267986,14027.746763
std,83.533884,11746.513305
min,8.000000,2850.000000
25%,82.000000,7890.000000
50%,105.000000,10490.000000
75%,136.000000,13950.000000
max,717.000000,78899.000000


In [78]:
data.to_csv('car_data.csv', index=False)